# 🌱 #30DayMapChallenge - Day 27: Micromapping

**Map for day 27 of the [#30DayMapChallenge](https://30daymapchallenge.com/) 2024, created for Náttúrufræðistofnun.**

---

### English
**Struthiopteris fallax Surfing the Temperature Wave**

*Struthiopteris fallax* is Iceland’s very own hot-spring-loving fern! Found only at **Deildartunguhver**, this tiny plant thrives in the warm, steamy soil near Iceland’s famous hot spring. Despite its cozy habitat, it is considered **endangered in Iceland**, with a very limited population.

This project maps the **micro-temperature landscape** around Deildartunguhver, showing how *S. fallax* “surfs” the geothermal heat. Data collected in **2021** from a **100 m² grid** records each plant’s location and temperature, revealing how this unique fern survives its intense, heated home.

📍 **To learn more**: [Click here](https://www.ni.is)

**#30DayMapChallenge #Micromapping #Iceland #Geothermal #StruthiopterisFallax #Mapping**

### How to Use
Try the notebook to explore Icelandic roundabouts or customize it for your own region!

| Run the code via free cloud platforms: | [![Binder](https://mybinder.org/badge.svg)](https://mybinder.org/v2/gh/lmi/30DayMapChallenge/master?filepath=/Day-27/Day27-MicroMapping.ipynb) | [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/lmi/30DayMapChallenge/blob/master/Day-27/Day27-MicroMapping.ipynb) | [![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lmi/30DayMapChallenge/blob/master/Day-27/Day27-MicroMapping.ipynb) |
|---|---|---|---|


---

### Data Sources
- 🌍 **Data Collection**: Pawel Wasowicz and Náttúrufræðistofnun  
- 🌡️ **Temperature and Plant Locations**: 2021 field survey, 100 m² grid at Deildartunguhver  

---

### Design Credits
- **Author**: Marco Pizzolato
- **Data Visualization Tools**: Python, Plotly 

---

**License**: Creative Commons **CC BY 4.0**

---

This map highlights the remarkable adaptation of *Struthiopteris fallax* to its geothermal environment and the delicate balance needed to protect this unique Icelandic species.

![Day 27 - Micromapping](Day27-MicroMapping_EN.png)

## Import and install

In [1]:
!pip install plotly


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.interpolate import griddata

## Load and process data

In [3]:
# Load the data
temperature_data = pd.read_csv('interpolated_temperature_10cm.csv')
individuals_data = pd.read_csv('individuals.csv')

In [4]:
# Interpolate temperature values for plant locations
individuals_data['Temperature'] = griddata(
    (temperature_data['X'], temperature_data['Y']),
    temperature_data['Interpolated_Temp_10cm'],
    (individuals_data['X'], individuals_data['Y']),
    method='linear'
)

## Plot

In [7]:
# Pivot the temperature data for structured grid needed for surface plot
temp_pivot = temperature_data.pivot(index="Y", columns="X", values="Interpolated_Temp_10cm")
X = temp_pivot.columns.values
Y = temp_pivot.index.values
X, Y = np.meshgrid(X, Y)
Z = temp_pivot.values  # The temperature values

# Define a custom color scale that goes from blue to red around the 40°C threshold
custom_colorscale = [
    [0, '#003aa0'],     # Low end of the scale (below 40°C, blue)
    [0.5, '#ffe877'],  # Midpoint around 40°C (neutral)
    [1, '#c50000']       # High end of the scale (above 40°C, red)
]

# Mask out boundary areas (setting NaNs to a baseline)
Z_masked = np.where(np.isnan(Z), 0, Z)  # Setting NaNs to baseline for visualization masking

# Create the interactive 3D plot using Plotly
fig = go.Figure()

fig.add_trace(go.Surface(
    x=X, y=Y, z=Z_masked,
    colorscale=custom_colorscale,
    colorbar=dict(
        title=dict(
            text='T (°C)',
            font=dict(size=20)  # Increase title font size
        ),
        tickfont=dict(size=20),  # Increase tick font size
        orientation='h',  # Horizontal orientation
        x=0.5,            # Center horizontally
        y=-0.15,          # Position below the plot
        xanchor='center', # Anchor horizontally at the center
        yanchor='top',    # Anchor vertically at the top
    ),
    opacity=0.7,
    showscale=True,
    cmin=0,
    cmax=np.nanmax(Z),
    hovertemplate="X: %{x}<br>Y: %{y}<br>T: %{z} °C",
    name='',
))



# Add individual plant locations as points with adjusted temperature, using dark green color
fig.add_trace(go.Scatter3d(
    x=individuals_data['X'],
    y=individuals_data['Y'],
    z=individuals_data['Temperature'],
    mode='markers',
    marker=dict(size=8, color='#295e2a'),  # Dark green color for the points
    name='Fern',
    hovertemplate="X: %{x}<br>Y: %{y}<br>T: %{z} °C"
))

# Set background color
b_color = "#a8d5ba"

# Update figure to remove axis and set limits
fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False, range=[np.min(X), 690]),  # Limiting X to max 690
        yaxis=dict(visible=False, range=[-100, np.max(Y)]),  # Limiting Y to min -100
        zaxis=dict(visible=False),  # Hiding Z axis
        bgcolor=b_color,  # Set the scene background color

        # Set the initial camera view
        camera=dict(
            eye=dict(x=1.5, y=-1.5, z=1.5),  # Controls zoom and rotation
            center=dict(x=0, y=0, z=0),     # Sets the center point of rotation
            up=dict(x=0, y=0, z=1)          # Defines the upward direction (z-axis is up)
        )
    ),
    paper_bgcolor=b_color,  # Set the background color outside the plot
    plot_bgcolor=b_color,   # Set the plot area background color
)

# Show the plot
fig.show()

## Make 3D plot html

In [8]:
import plotly.io as pio

# Export the figure using plotly.io
pio.write_html(fig, "map.html", auto_play=False)

## Zip files

In [32]:
import os
import zipfile

# Define the output zip file name
output_zip_path = "fern.zip"

# Get a list of all files in the current working directory
folder_to_zip = os.getcwd()

# Make list of files to zip without the zip itself
files_to_zip = [
    os.path.join(folder_to_zip, f)
    for f in os.listdir(folder_to_zip)
    if os.path.isfile(os.path.join(folder_to_zip, f)) and f != output_zip_path
]

# Create a new zip file and add each file in the list
with zipfile.ZipFile(output_zip_path, 'w') as zipf:
    for file in files_to_zip:
        zipf.write(file, arcname=os.path.basename(file))  # Use arcname to save files without full paths

print(f"Zip file created with all files from the current directory: {output_zip_path}")

Zip file created with all files from the current directory: fern.zip
